In [1]:
import cv2
import face_recognition
import pickle
import numpy as np

# Load the saved face model
with open('trained_face_model.pkl', 'rb') as model_file:
    model_data = pickle.load(model_file)
    encodings_list = model_data['encodings']
    labels_list = model_data['labels']
    label_dict = model_data['label_dict']

# Initialize OpenCV's face detector (Haar Cascade)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize webcam
cap = cv2.VideoCapture(0)  # Change to 1 if using an external camera

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

print("Press 'q' to exit.")
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to grab frame.")
        break

    # Convert frame to grayscale for faster processing (Haar Cascade requires grayscale images)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale image
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Loop over the detected faces
    for (x, y, w, h) in faces:
        # Draw rectangle around face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Extract the region of interest (face) from the frame
        face_roi = frame[y:y + h, x:x + w]
        
        # Convert to RGB (face_recognition uses RGB, OpenCV uses BGR)
        rgb_face = cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB)

        # Find face encodings for the detected face
        face_locations = [(0, w, h, 0)]  # Fake location as we already have the ROI
        face_encodings = face_recognition.face_encodings(rgb_face, face_locations)

        # Loop through all the faces found
        for face_encoding in face_encodings:
            # Compare the detected face encoding with saved encodings
            matches = face_recognition.compare_faces(encodings_list, face_encoding)
            name = "Unknown"
            confidence = 0.0  # Default confidence

            # If a match is found, get the name and calculate the confidence
            if True in matches:
                first_match_index = matches.index(True)
                name = label_dict[labels_list[first_match_index]]

                # Calculate the distance between the detected face and the stored face encoding
                face_distances = face_recognition.face_distance([encodings_list[first_match_index]], face_encoding)
                confidence = 1 - face_distances[0]  # Convert distance to confidence (higher is better)

            # Show the name and confidence level on the frame
            cv2.putText(frame, f"{name} ({confidence*100:.2f}%)", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the video feed with face rectangles, names, and confidence
    cv2.imshow("Real-time Face Recognition", frame)

    # Exit when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close OpenCV window
cap.release()
cv2.destroyAllWindows()


Press 'q' to exit.


2024-11-28 10:31:15.335 python3.12[57210:1840029] +[IMKClient subclass]: chose IMKClient_Legacy
2024-11-28 10:31:15.335 python3.12[57210:1840029] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


KeyboardInterrupt: 